In [ ]:
import ee
# from geemap import geemap
import pandas as pd
import geemap.plotlymap as geemap

In [ ]:
ee.Authenticate()
ee.Initialize(project='sentinel-treeclassification')

london_trees_df = pd.read_csv("data/Borough_tree_list_2021July.csv")

In [ ]:
def mask_s2_clouds(image):
  # Quality assessment with resolution in meters
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask)


dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2020-01-01', '2020-01-30')
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
)

# tci_bands = ['TCI_R', 'TCI_G', 'TCI_B']
# tci_max = 0.02550000138580799
rgb_max = 3000
rgb_bands = ['B4', 'B3', 'B2']
visualization = {
    'min': 0.0,
    'max': rgb_max,
    'bands': rgb_bands,
}

m = geemap.Map()
m.set_center(51.5, 0.1, 12)
m.addLayer(dataset.mean(), visualization, 'RGB')

In [ ]:
# m = m.add_scattergeo(lat=london_trees_df['latitude'], lon=london_trees_df['longitude'])

In [ ]:
# import plotly.io as pio
# pio.renderers.default = "vscode"
# m.show()
m

In [ ]:
poi = ee.Geometry.Point([83.277, 17.7009])
dataset.select("B2").max().reduceRegion(ee.Reducer.first(), poi, 30)

In [ ]:
pixel_side = 1000
buffer_meters = 10*pixel_side/2 - 10
region = poi.buffer(buffer_meters).bounds()
m.layer_to_image('RGB', output=f'sample_{pixel_side}_pixelside.png', region=region, scale=10)


In [ ]:
clipped_image = dataset.mean().clip(region)
m = geemap.Map()
m.set_center(83.277, 17.7009, 12)
m.add_layer(clipped_image, visualization, 'RGB')
m

In [ ]:
clipped_image.select('B2') 

In [ ]:
# A Sentinel-2 surface reflectance image, reflectance bands selected,
# serves as the source for training and prediction in this contrived example.
img = ee.Image(
    'COPERNICUS/S2_SR/20210109T185751_20210109T185931_T10SEG'
).select('B.*')

# ESA WorldCover land cover map, used as label source in classifier training.
lc = ee.Image('ESA/WorldCover/v100/2020')

# Remap the land cover class values to a 0-based sequential series.
class_values = [10, 20, 30, 40, 50, 60, 70, 80, 90, 95, 100]
remap_values = ee.List.sequence(0, 10)
label = 'lc'
lc = lc.remap(class_values, remap_values).rename(label).toByte()

# Add land cover as a band of the reflectance image and sample 100 pixels at
# 10 m scale from each land cover class within a region of interest.
roi = ee.Geometry.Rectangle(-122.347, 37.743, -122.024, 37.838)
sample = img.addBands(lc).stratifiedSample(
    numPoints=100, classBand=label, region=roi, scale=10, geometries=True
)

# Add a random value field to the sample and use it to approximately split 80%
# of the features into a training set and 20% into a validation set.
sample = sample.randomColumn()
training_sample = sample.filter('random <= 0.8')
validation_sample = sample.filter('random > 0.8')

# Train a 10-tree random forest classifier from the training sample.
trained_classifier = ee.Classifier.smileRandomForest(10).train(
    features=training_sample,
    classProperty=label,
    inputProperties=img.bandNames(),
)

# Get information about the trained classifier.
display('Results of trained classifier', trained_classifier.explain())

# Get a confusion matrix and overall accuracy for the training sample.
train_accuracy = trained_classifier.confusionMatrix()
display('Training error matrix', train_accuracy)
display('Training overall accuracy', train_accuracy.accuracy())

# Get a confusion matrix and overall accuracy for the validation sample.
validation_sample = validation_sample.classify(trained_classifier)
validation_accuracy = validation_sample.errorMatrix(label, 'classification')
display('Validation error matrix', validation_accuracy)
display('Validation accuracy', validation_accuracy.accuracy())

# Classify the reflectance image from the trained classifier.
img_classified = img.classify(trained_classifier)

# Add the layers to the map.
class_vis = {
    'min': 0,
    'max': 10,
    'palette': [
        '006400',
        'ffbb22',
        'ffff4c',
        'f096ff',
        'fa0000',
        'b4b4b4',
        'f0f0f0',
        '0064c8',
        '0096a0',
        '00cf75',
        'fae6a0',
    ],
}
m = geemap.Map()
m.set_center(-122.184, 37.796, 12)
m.add_layer(
    img, {'bands': ['B11', 'B8', 'B3'], 'min': 100, 'max': 3500}, 'img'
)
m.add_layer(lc, class_vis, 'lc')
m.add_layer(img_classified, class_vis, 'Classified')
m.add_layer(roi, {'color': 'white'}, 'ROI', False, 0.5)
m.add_layer(training_sample, {'color': 'black'}, 'Training sample', False)
m.add_layer(
    validation_sample, {'color': 'white'}, 'Validation sample', False
)
m

In [ ]:
sample